In [4]:
import os
from glob import glob
import pandas as pd
import numpy as np
from PIL import Image

In [5]:
# You first need to set you directory structure
# and collect the behavioral files for the localizer and the
# study task separately.  Given that each task will be modeled
# separately treat them separately.
# use os.path.join or Pathlib to define location of files
# use glob and sort to grab relevant files...I would separately handle localizer and task .tsv files

# My directory on my local computer
proj_dir = '/Users/alisha/Desktop/CogNeuro-Imaging-Methods/Mattfeld_PSB6351/behav'

# Used glob and * wild card element like previous assignment to locate localizer and study task files
localizer_files = sorted(glob(os.path.join(proj_dir, '*_task-loc_*_events.tsv')))
study_task_files = sorted(glob(os.path.join(proj_dir, '*_task-study_*_events.tsv')))

# Print files to make sure the path is correct
print("Localizer files:", localizer_files)
print("Study task files:", study_task_files)

Localizer files: ['/Users/alisha/Desktop/CogNeuro-Imaging-Methods/Mattfeld_PSB6351/behav/sub-021_task-loc_run-1_events.tsv', '/Users/alisha/Desktop/CogNeuro-Imaging-Methods/Mattfeld_PSB6351/behav/sub-021_task-loc_run-2_events.tsv']
Study task files: ['/Users/alisha/Desktop/CogNeuro-Imaging-Methods/Mattfeld_PSB6351/behav/sub-021_task-study_run-1_events.tsv', '/Users/alisha/Desktop/CogNeuro-Imaging-Methods/Mattfeld_PSB6351/behav/sub-021_task-study_run-2_events.tsv', '/Users/alisha/Desktop/CogNeuro-Imaging-Methods/Mattfeld_PSB6351/behav/sub-021_task-study_run-3_events.tsv', '/Users/alisha/Desktop/CogNeuro-Imaging-Methods/Mattfeld_PSB6351/behav/sub-021_task-study_run-4_events.tsv']


In [6]:
# Define variables that can distinguish between runs and have 
# onset times and their duration embedded. I used dictionaries.
# This will need to be accomplished for each type of stimulus or
# regressor you want to isolate onset times for.

# Prepare dictionaries to hold the onset times for each condition
loc_face_onset_times = {'run1': [], 'run2': []}
loc_scence_onset_times = {'run1': [], 'run2': []}
loc_math_onset_times = {'run1': [], 'run2': []}

# The localizer_files variable is already defined in the first part of the code, using glob to find relevant files.
# Here we proceed with this dynamically generated list.
for idx, curr_behav_file in enumerate(localizer_files):  
    # Read in the log files
    curr_behav_data = pd.read_csv(curr_behav_file, sep='\t')
    
    # Temporary lists to grab all onset times for each condition
    tmp_face_onset = []
    tmp_scence_onset = []
    tmp_math_onset = []
    
    # Iterating over trial types
    for i, curr_trial_type in enumerate(curr_behav_data['trial_type']):  
        if curr_trial_type == 'face':  # Checking for face stimuli
            tmp_face_onset.append(curr_behav_data['onset'][i])
        elif curr_trial_type == 'scence':  # Checking for scence stimuli
            tmp_scence_onset.append(curr_behav_data['onset'][i])
        elif curr_trial_type == 'math':  # Checking for math stimuli
            if len(tmp_face_onset) > 0:  
                loc_face_onset_times[f'run{idx + 1}'].append(f"{tmp_face_onset[0]}:25")
                tmp_face_onset = []  # Reset temporary variable for next face block
            
            if len(tmp_scence_onset) > 0:  
                loc_scence_onset_times[f'run{idx + 1}'].append(f"{tmp_scence_onset[0]}:25")
                tmp_scence_onset = []  # Reset temporary variable for next scence block
                
            if len(tmp_math_onset) == 0:  
                tmp_math_onset.append(curr_behav_data['onset'][i])  
            
            if len(tmp_math_onset) > 0:  
                loc_math_onset_times[f'run{idx + 1}'].append(f"{tmp_math_onset[0]}:25")
                tmp_math_onset = []  # Reset for the next math block

# Convert lists to comma-separated strings for output
loc_scence_run1_data = ", ".join(map(str, loc_scence_onset_times['run1']))
loc_scence_run2_data = ", ".join(map(str, loc_scence_onset_times['run2']))
loc_face_run1_data = ", ".join(map(str, loc_face_onset_times['run1']))
loc_face_run2_data = ", ".join(map(str, loc_face_onset_times['run2']))
loc_math_run1_data = ", ".join(map(str, loc_math_onset_times['run1']))  # Correct
loc_math_run2_data = ", ".join(map(str, loc_math_onset_times['run2']))

# Define the sink directory where to save the timing files
evs_sink_dir = os.path.join(proj_dir, 'EV_files_output')
if not os.path.isdir(evs_sink_dir):
    os.makedirs(evs_sink_dir)

# Define and save the files for the localizer task
loc_scence_evs_file = 'loc_scence_evs.1D'
with open(os.path.join(evs_sink_dir, loc_scence_evs_file), 'wt') as fp:
    fp.writelines([f'{loc_scence_run1_data}\n'])
    fp.writelines([f'{loc_scence_run2_data}\n'])

loc_face_evs_file = 'loc_face_evs.1D'
with open(os.path.join(evs_sink_dir, loc_face_evs_file), 'wt') as fp:
    fp.writelines([f'{loc_face_run1_data}\n'])
    fp.writelines([f'{loc_face_run2_data}\n'])

loc_math_evs_file = 'loc_math_evs.1D'
with open(os.path.join(evs_sink_dir, loc_math_evs_file), 'wt') as fp:
    fp.writelines([f'{loc_math_run1_data}\n'])
    fp.writelines([f'{loc_math_run2_data}\n'])

# Print statements to check the output
print("Face onset times:", loc_face_onset_times)
print("Scence onset times:", loc_scence_onset_times)
print("Math onset times:", loc_math_onset_times)
print("Scence Run 1 Data:", loc_scence_run1_data)
print("Scence Run 2 Data:", loc_scence_run2_data)
print("Face Run 1 Data:", loc_face_run1_data)
print("Face Run 2 Data:", loc_face_run2_data)
print("Math Run 1 Data:", loc_math_run1_data)
print("Math Run 2 Data:", loc_math_run2_data)

Face onset times: {'run1': ['6.01005580311:25', '81.0102012303:25', '156.010947896:25', '231.012253435:25', '306.02929695:25', '381.030623221:25', '456.031369286:25'], 'run2': ['6.010115296:25', '81.0108769855:25', '156.012685209:25', '231.012760026:25', '306.014053246:25', '381.014985602:25', '456.015892418:25']}
Scence onset times: {'run1': ['43.5105377558:25', '118.509990601:25', '193.51066846:25', '268.528655448:25', '343.529607034:25', '418.530665587:25', '493.531335934:25'], 'run2': ['43.5103616808:25', '118.510910338:25', '193.512537979:25', '268.513247087:25', '343.514315555:25', '418.515630709:25', '493.516578389:25']}
Math onset times: {'run1': ['31.0005110982:25', '32.2505222155:25', '33.5003924128:25', '34.7504756427:25', '36.0005450512:25', '37.2506934829:25', '38.5008551354:25', '39.7507088069:25', '41.001029107:25', '42.2506505159:25', '68.5006691449:25', '69.7509666095:25', '71.0006451074:25', '72.250683868:25', '73.5007514735:25', '74.7506994923:25', '76.0007139149:25'

In [20]:
# In this cell I'm going to first work on the localizer task

# Define variables that can distinguish between runs and have 
# onset times and their duration embedded.  I used dictionaries
# This will need to be accomplished for each type of stimulus or
# or regressor you want to isolate onset times for

# Iterate over your text files for the localizer task.
# The variable curr_behav_file will be a string variable with
# the full path to the separate runs of the localizer task. idx is a counter
# used for indexing.
for idx, curr_behav_file in enumerate(name_of_variable_for_localizer_files_defined_in_previous_cell):
    # This is where I defined the keys in my dictionaries that were created above
    # and defined the output associated with each key as an empty list
    # example --> name_of_dictionary_defined_above[f'run{SOMETHING}'] = []
    
    # I'm using the pandas function read_csv to read in the log files
    curr_behav_data = pd.read_csv(curr_behav_file, sep='\t')
    
    # Given that the localizer task is a block design you don't need 
    # the onset time of each stimulus.  You just need the first and then
    # the total duration of all stimuli to create your blocked time
    # to convolve with your hemodynamic response.  I used temporary lists
    # to grab all onset times and then picked the first...you can choose
    # another way.
    
    # iterating over ????? here...i is counter for indexing
    # What column header would you want to iterate over?  Why?
    for i, curr_trial_type in enumerate(curr_behav_data['?????']):
        if curr_trial_type == '????': #What would you want to look for in this conditional statement?
            # Here I am appending the onset of the stimulus if the current
            # trial type is a ??????.
            # I used a list append function here to grow my temporary list variable
            # reference above face-by-face...you can choose a different way
            # example --> name_of_temp_variable.append(curr_behav_data['????'][i]) #What is i representing? 
        elif curr_trial_type == '?????': #note...scence was misspelled originally
            # Here I am appending the onset of the stimulus if the current
            # trial type is a scene.
            # I used a list append function here to grow my temporary list variable
            # reference above face-by-face...you can choose a different way
            
        # here I am using the first trial type when it becomes math and the 
        # face onset list variable is ???? elements long (just exited a face block)
        # to assign the first element of the tmp_face_onset list variable to the 
        # dictionary that I created earlier.
        # the format of AFNI stimulus timing files is the following:
        #
        # onset_time:duration (e.g., 6.3:25, 12.7:25, 22.5:25) --> A block started at 6.3, 12.7, and 22.5 seconds
        # into the experiment and each block lasted for 25 seconds in length...make sure these numbers match your
        # experimental design
        elif curr_trial_type == 'math' and len(tmp_face_onset) == ?????:
            # This is where I update my run dictionary with the current onset time and duration of 
            # the block of stimuli of a specific condition....HINT: 0 is the first in a list of elements
            # Be sure to reset your temporary variable as you will be now collecting a new set
            # of onset times for your new block of stimuli
        elif curr_trial_type == 'math' and len(tmp_scene_onset) == ?????:
            # Do the same as you did above but for the different stimuli conditions.
            
# The following code creates a string element that has the square brackets
# removed.  This is important for the following steps below.
# I'm leaving this for you...If you opt to not use dictionaries like I did then
# you'll have to find a different way.
loc_scene_run1_data = ", ".join(map(str, loc_scene_onset_times['run1']))
loc_scene_run2_data = ", ".join(map(str, loc_scene_onset_times['run2']))
loc_face_run1_data = ", ".join(map(str, loc_face_onset_times['run1']))
loc_face_run2_data = ", ".join(map(str, loc_face_onset_times['run2']))

# Here I am defining the sink directory where I would like to save the timing files
# HINT you shouldn't save these text files in teh same directory where your code resides
# you can also you Pathlib to create this and make the direcotry if it doesn't exist
evs_sink_dir = os.path.join(proj_dir, 'where', 'do', 'you', 'want', 'these', 'files', 'saved')
# I check to see if the directory exists.  If it doesn't I create it.
if not os.path.isdir(evs_sink_dir):
    os.makedirs(evs_sink_dir)
    
# below I am defining the file names for the localizer (loc) face and scene evs.
# each run is captured on a separate line with the multiple onsets within a run
# captured on a single line
# I'm leaving this for you...again this may constrain the way you do things above
# but it gives you a hint of how to output data into a AFNI style format
# your output should eventually look like the following:
#
# 6.3:25, 30.7:25, 76.5:25, 137.3:25, 225.1:25
# 6.3:25, 30.7:25, 76.5:25, 137.3:25, 225.1:25
#
# The first line refers to run1 and the second line refers to run2
# in this case the onset times were identical for this particular condition across runs
loc_scene_evs_file = 'loc_scene_evs.1D'
with open(os.path.join(evs_sink_dir, loc_scene_evs_file), 'wt') as fp:
    fp.writelines([f'{loc_scene_run1_data}\n'])
    fp.writelines([f'{loc_scene_run2_data}\n'])
loc_face_evs_file = 'loc_face_evs.1D'
with open(os.path.join(evs_sink_dir, loc_face_evs_file), 'wt') as fp:
    fp.writelines([f'{loc_face_run1_data}\n'])
    fp.writelines([f'{loc_face_run2_data}\n'])

In [21]:
# Similar to above I am creating empty dictionary variables
# for each of the events that I am interested in.
# I will then insert run keys to separate the timing files 
# for the events of interest and their specific runs.

# Here I am iterating over the study behavior files.  There should be
# 4 of them.
for idx, curr_behav_file in enumerate(name_of_variable_for_study_files_defined_in_previous_cell):
    # I set the run key for each condition of interest
    # Again if you opt not to use the dictionaries as I did
    # you'll need to utilize a different way. HINT you should
    # update the key for as many events of interest you are
    # attempting to isolate
    
    # I read in the current study run behavioral file
    # NOTE: sometimes it is not good practice to use the same
    # variable names across cells. It may execute but erroneously
    curr_behav_data = pd.read_csv(curr_behav_file, sep='\t')
    
    # I iterate now over the contents of the run specific data.
    # Again...you'll want to isolate the column header that lets
    # you assess the event or trial types of interest
    for i, curr_trial_type in enumerate(curr_behav_data['??????']):
        # I am evaluating whether or not the current trail type was a 
        # fixed association that had a conditional trial that followed with a face
        # or a scene
        if '??????' in curr_trial_type or '?????' in curr_trial_type:
            # if it was either of those grab that onset
            tmp_fix_onset = GRAB THE ONSET WITH THE APPROPRIATE CODE
            # if this is not our first trial (i = counter > 0) - remember python is 0-based
            if i > 0:
                # evaluate whether or not the LAST TRIAL (i-1) was a scene or face fix trial
                # grab the current onset time and assign it to the remaining events.
                # In the analysis that I am interested in pursuing I want to separate these
                # trials from trials where the fix face and scence trials are followed either
                # by a conditional trial or by a baseline trial
                
                if '?????' in curr_behav_data['trial_type'][i-1] or '??????' in curr_behav_data['trial_type'][i-1]:
                    task_remain_events_onset_times[f'run{idx+1}'].append(curr_behav_data['onset'][i])
        # Here I am evaluating whether or not the current trial type is a conditional trial 
        # that was responded to correctly
        elif # grab the onset times for fixed association trials before correct and incorrect conditional trials
             # this will require multiple elif statements.  You'll have to assess what the curr_trial_type is
             # and whether or not that trial was correct or incorrect.  You'll then input that 'tmp_fix_onset'
             # variable that you held onto when you evaluated the above trial types
        # Now I am doing the same thing for trials that precede the perceptual baseline trials
        # These trials will be used for the MVPA anlaysis that is planned.
        elif curr_trial_type == 'baseline' and '?????' in curr_behav_data['trial_type'][i-1]:
                task_facefixb4_bl_onset_times[f'run{idx+1}'].append(tmp_fix_onset)
        
            
# Given that we're setting things up to analyze in AFNI
# you can't have runs that don't have any events in them....or
# you can but you can't have an empty row...thus here I am checking
# to see if the runs are empty and if they are adding a filler (-1)
for curr_run in ['run1', 'run2', 'run3', 'run4']:
    if len(NAME_OF_DICT_FOR_STIMULUS_CONDITION_OF_INTEREST[curr_run]) == 0:
        NAME_OF_DICT_FOR_STIMULUS_CONDITION_OF_INTEREST[curr_run].append(-1)
    # the code above would have to be repeated for each variable that is 
    # capturing a stimulus condition of interest. THis is not the most
    # effecient way of coding and can be improved upon.

# Reformatting as before to save as a text file with no square brackets
# use what you did for the localizer task here. If you use the same way as before
# or that I used you'll have to have a separate line for each run of each condition
# This is not a great way to code and could be improved upon

# Creating and checking to see if the directory exists for where you want to save your text files
# use the way you did this for the localizer task


# creating my separate ev files with runs written to each line
# use the way you did this before. HINT if you keep the same way as before
# and how I did it you'll need a separate block of code for each regressor (aka stimulus condition)
# you are trying to model and a separate line within each block for each run
# In the end it should look something like the following in one file:
#
# 6.3, 30.7, 76.5, 137.3, 225.1
# 6.3, 30.7, 76.5, 137.3, 225.1
# 6.3, 30.7, 76.5, 137.3, 225.1
# 6.3, 30.7, 76.5, 137.3, 225.1
# 
# In the example above this stimulus had the exact same onset times across each runs
# your output should in format look like this but your onset times should not be the
# same across runs!!!!



In [24]:
# This cell is required to run the following 3dDeconvolve calls in the subsequent cells
# Here I am collecting the motion related regressor files that were created for each run
# separately, concatenating them and saving the output.

# the below directory structure will change to match how your data are organized
proj_dir = '/home/amattfel/Mattfeld_PSB6351/'
motion_dir = '/derivatives/preproc/sub-021/motion1stAFNI'

loc_motion_files = sorted(glob(proj_dir + motion_dir + '/st*/*loc*bold.1D'))
loc_mot_dict = {}
for mot_i, curr_loc_motion_file in enumerate(loc_motion_files):
    loc_mot_dict[f'run{mot_i + 1}'] = np.genfromtxt(curr_loc_motion_file)
    
allruns_loc_motion_data = np.concatenate((loc_mot_dict['run1'], loc_mot_dict['run2']))
np.savetxt(proj_dir + motion_dir + '/allruns_loc_mot_data.1D', allruns_loc_motion_data)

task_motion_files = sorted(glob(proj_dir + motion_dir + '/st*/*study*bold.1D'))
task_mot_dict = {}
for mot_i, curr_task_motion_file in enumerate(task_motion_files):
    task_mot_dict[f'run{mot_i + 1}'] = np.genfromtxt(curr_task_motion_file)
    
allruns_task_motion_data = np.concatenate((task_mot_dict['run1'], task_mot_dict['run2'],
                                           task_mot_dict['run3'], task_mot_dict['run4']))
np.savetxt(proj_dir + motion_dir + '/allruns_task_mot_data.1D', allruns_task_motion_data)



In [ ]:
%%bash
# Given that we are running this with no data (see -nodata flag) we can run a quick bash
# command in the cell by using line above.  The matrices that are created and image
# will be created in the directory where this jupyter notebook is running. In my case.
# /home/data/madlab/Mattfeld_PSB6351/mattfeld_2020/code

3dDeconvolve -nodata 608 1.76 \
-concat '1D: 0 304' \
-ortvec /home/amattfel/Mattfeld_PSB6351/derivatives/preproc/sub-021/motion1stAFNI/allruns_loc_mot_data.1D motion \
-polort A \
-local_times \
-num_stimts 2 \
-stim_times_AM1 1 /home/amattfel/Mattfeld_PSB6351/derivatives/first_lvl/sub-021/evs/loc_face_evs.1D "dmBLOCK(1)" -stim_label 1 faces \
-stim_times_AM1 2 /home/amattfel/Mattfeld_PSB6351/derivatives/first_lvl/sub-021/evs/loc_scene_evs.1D "dmBLOCK(1)" -stim_label 2 scenes \
-x1D X.loc.xmat.1D -xjpeg X.loc.jpg


In [ ]:
%%bash

3dDeconvolve -nodata 1420 1.76 \
-concat '1D: 0 355 710 1065' \
-ortvec /home/amattfel/Mattfeld_PSB6351/derivatives/preproc/sub-021/motion1stAFNI/allruns_task_mot_data.1D motion \
-polort A \
-local_times \
-num_stimts 5 \
-stim_times 1 /home/amattfel/Mattfeld_PSB6351/derivatives/first_lvl/sub-021/evs/fix_b4_c_cond_evs.1D "TWOGAMpw(4,5,0.2,12,7)" -stim_label 1 fx_b4_c_cond \
-stim_times 2 /home/amattfel/Mattfeld_PSB6351/derivatives/first_lvl/sub-021/evs/fix_b4_ic_cond_evs.1D "TWOGAMpw(4,5,0.2,12,7)" -stim_label 2 fx_b4_ic_cond \
-stim_times 3 /home/amattfel/Mattfeld_PSB6351/derivatives/first_lvl/sub-021/evs/events_remain_evs.1D "TWOGAMpw(4,5,0.2,12,7)" -stim_label 3 all_remain \
-stim_times 4 /home/amattfel/Mattfeld_PSB6351/derivatives/first_lvl/sub-021/evs/facefix_b4_bl_evs.1D "TWOGAMpw(4,5,0.2,12,7)" -stim_label 4 facefx_b4_bl \
-stim_times 5 /home/amattfel/Mattfeld_PSB6351/derivatives/first_lvl/sub-021/evs/scenefix_b4_bl_evs.1D "TWOGAMpw(4,5,0.2,12,7)" -stim_label 5 scenefx_b4_bl \
-x1D X.task.xmat.1D -xjpeg X.task.jpg


In [ ]:
im_task = Image.open(os.path.join(os.getcwd(), 'X.task.jpg'))
im_task

In [ ]:
im_loc = Image.open(os.path.join(os.getcwd(), 'X.loc.jpg'))
im_loc